<a href="https://colab.research.google.com/github/Ashish-Soni08/SQL-Code-Generation-Application/blob/master/week_1/LLMS_In_Production_W1_Project_FINAL_Ashish_Soni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Week 1 and DataAlchemy Labs as a Founding Machine Learning Engineer!

As a founding Machine Learning Engineer on the team you and your team have narrowed down a life-changing product. You will be building and launching your groundbreaking SaaS product which is quite similar to a product already out in the market, [AI2sql](https://www.ai2sql.io/).


![AI2sql Example](https://drive.google.com/uc?id=1mWOQs6OZmuCLui-auLosXsXGhwl2mbVg)

You have done a significant amount of market research and are confident that this product will gather a cult-like following with a killer feature of being able to take any natural language query and output SQL code based on the query.

Potential users of your application are:
- **Project Managers**: They can use our tool to quickly derive important metrics for presentations or reports. Instead of relying on technical teams, they can directly query the database using natural language.
- **Data Scientists**: Often dealing with complex data queries, data scientists can use our tool to streamline and debug intricate SQL statements, saving time and reducing the potential for error.
- **Business Analysts and Non-Tech Professionals**: Anyone without deep SQL knowledge but needing to interact with the database can benefit immensely. They can easily convert their data needs into SQL queries without delving into the complexities of SQL syntax.

There are many more features that you can work on in the future such as:
1. NoSQL Code Generation
2. SQL Syntax Checking
3. Explaining SQL Code
4. Optimizing SQL Code
5. Formatting SQL Code

But since this is an MVP and you want to laucnh as quickly as possible we will be just focussing on the use case of SQL Code Generation. In particular we will be supporting the use of **SQL** initially as its the most popular database out there in use currently based on the recent [StackOverflow Developer Survey 2023](https://survey.stackoverflow.co/2023/#section-most-popular-technologies-databases).



Our final product will be a simple web application with a basic Q/A system that allows users to ask query about SQL and then get back SQL code that they can use for their application, think of it as a super basic version of the ChatGPT from OpenAI that I am sure everyone is quite familiar with.

![ChatGPT OpenAI Example](https://drive.google.com/uc?id=1AGVZoxtWvBF6KLX0mkvpnU7mMEgaSraa)

In the Week 1 Project we will be solidfying what we learnt throughout the week by building out this core functionality for the SQL Code Generation part of our application. Just like all Data Science projects we start off in a notebook environment to prototype things before having to bring them to production.

Just like most LLM startups we will be leveraging OpenAI and their ChatGPT API initially for our product, in particular we will be using **gpt-3.5-turbo** which offers a good enough LLM that is quite fast!

The main parts that we will be covering in the project are:
1. Using LLMs to Generate SQL Code
2. Evaluating LLMs on SQL Code Generation
3. Validating LLM Outputs using Guardrails

At the end of the project we will highlight a couple of optional tasks that we encourage you to try out to really make the most out of the course, altough they aren't necessary to complete the course as some of them are quite challenging and time consuming.





# 1. Using LLMs to Generate SQL Code

## SQL Code Generation

SQL is one of the most powerful programming languages out there and is the main way in which we communicate with databases. Without it backend engineers would be unable to store and structure data to be easily queried to return to a frontend application and data scientists would be unable to track key metrics about their experiments over time.

SQL is the language used for interacting with databases, while SQL is a specific database system that understands and uses SQL for database operations. Each database system, including SQL, implements SQL with some variations and adds its own proprietary extensions to the standard SQL language.

Here is an example SQL statement:

```sql
SELECT
    u.name,
    u.email,
    COUNT(o.order_id) AS total_orders,
    SUM(o.amount) AS total_amount_spent
FROM
    users u
JOIN
    orders o ON u.user_id = o.user_id
WHERE
    o.order_date >= (NOW() - INTERVAL '1 year')
GROUP BY
    u.user_id
HAVING
    COUNT(o.order_id) > 5
ORDER BY
    total_amount_spent DESC;
```

LLM's such as ChatGPT from OpenAI are trained on all kinds of data across the internet and in particular they are trained on code across many different programming languages.


![Text2SQL](https://drive.google.com/uc?id=17IDU11L_JdPIfMWvDa6li_5oOyGgWriX)

What we will be performing is asking the LLM a Question related to a SQL query in order to create SQL code that we can then run.


During OpenAI DevDay, there was a fantastic breakout section about **"[A Survey of Techniques for Maximizing LLM Performance](https://youtu.be/ahnGLM-RC1Y?si=rifoSUxFgvgliFtT)"** which we recommend all of you go through. In that talk, the speakers highlighted how we should always be starting off with Prompt Engineering when building LLM applications. Once we understand what our LLM application lacks can we only try more complex techniques such as RAG or Fine-tuning or both. With that spirit in mind let's try out some basic Prompt Engineering using ChatGPT.

![Optimization Flow from OpenAI](https://drive.google.com/uc?id=1kcguLT8KYBmDypGB0fVyeT8YfrCiiMx_)

In [ ]:
# Install the following dependencies that we will need
# Please do restart the runtime after this step!
!pip install -q langchain
!pip install -q langchain-openai
!pip install -q openai
!pip install -q guardrails-ai
!pip install -q pydantic
!pip install -q deepeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('openai_api')

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import LLMResult, HumanMessage, Generation

MODEL_NAME = "gpt-3.5-turbo"
MAX_TOKENS = None

llm = ChatOpenAI(
    model_name=MODEL_NAME,
    max_tokens=MAX_TOKENS,
    temperature=0.0
)

## Running a Local LLM - LLAMA CPP(NOT USED IN THIS PROJECT)

If you are not able to create an OpenAI account and instead would like to use a local LLM, you can leverage a tool such as [llama.cpp](https://github.com/ggerganov/llama.cpp). This will obviouly have longer processing times but you will not be charged credits for using an API especially considering the fact that this method will need to be deployed, therefore you will need much more compute even if you decide to run the model on CPU.

We recommend checking the amount of memory/disk that will be needed for the LLM based on these [calculations](https://github.com/ggerganov/llama.cpp/tree/master?tab=readme-ov-file#memorydisk-requirements). It is recommended to use models that will consume `<10GB` of Memory as in the Week 2 project we will be using Github Codespaces that has a limit of `32GB of RAM` within the free tier.  

Please note that this approach is experimental and will be more variable in terms of quality of output and speed(it will be much slower due to it being run on CPU). The libraries and tools are experimental in nature and can change at short notice. Our recommendation would be to stick with the default options presented above for a simple approach.

But we do present these options that you can run for free within a Colab notebook. This is an alternative way of completing the project for those who have run out of credits on OpenAI or do not want to attach a credit card for any reason.

In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
    repo_id="Qwen/Qwen1.5-0.5B-Chat-GGUF",
    filename="*q8_0.gguf",
    verbose=False
)

## TODO: Create a Baseline Zero-Shot Prompt

Create a Zero-Shot Prompt that we can use in this situation, we have provided you with a simple prompt to try out. A **Zero-Shot prompt** is where we are asking the model to perform a task it has never seen explicty during training without any additional examples.

When working on the prompt or improving the prompt remember a couple of things:

1. Make sure to use clear instructions
2. Try splitting tasks into simpler subtasks
3. Give the model time to “think”

![How to Prompt Engineer](https://drive.google.com/uc?id=1iSZ6XvD5eMiRSQBUT0DFEDA-1qc0qibv)


In [ ]:
# TODO: Add your prompt here, make sure to use
# {query} to inject a query into the prompt.

ZERO_SHOT_PROMPT_TEMPLATE = """

# ROLE

Act as an Expert SQL Code Generator, specializing in taking a natural language {query} from a user and generating SQL Code based on the {query}.

# Context

You help users from a non-technical and a technical background to generate SQL code to get answers for their data queries faster in a reliable way.

Potential users are:
- **Project Managers**: They can use our tool to quickly derive important metrics for presentations or reports. Instead of relying on technical teams, they can directly query the database using natural language.
- **Data Scientists**: Often dealing with complex data queries, data scientists can use our tool to streamline and debug intricate SQL statements, saving time and reducing the potential for error.
- **Business Analysts and Non-Tech Professionals**: Anyone without deep SQL knowledge but needing to interact with the database can benefit immensely. They can easily convert their data needs into SQL queries without delving into the complexities of SQL syntax.

# RESPONSIBILITY

Generate SQL Code that helps the users with their Data needs to fulfill any requirements they have.
Create posts from “Quotes” that compels to audience to stop and gaze at the stunning image and as they whisper the quote to themselves and read the story, they smile and cannot help but feel motivated, inspired, and grateful as they come to acknowledge the mental resilience needed to go through life.

# INSTRUCTIONS
1. **Input:** User will enter a Natural Language “Query” for a specific task
2. Take your time to understand the query
3. Generate well organized and optimized SQL Code with concise and clear comments that fulfills the user requirements

# RULES
1. Write a comment what the task is about and also add your desgination - written by SQLWhiz🤖 on top of the generated SQL Code.
2. You only answer natural language queries in 'English'
3. The final output should be the SQL Code
"""

# Try your own query
SAMPLE_QUERY = "Select the name of the employee who has the highest salary"

prompt = ZERO_SHOT_PROMPT_TEMPLATE.format(
    query=SAMPLE_QUERY
)

result = llm.generate([[HumanMessage(content=prompt)]])
# Do try to investigate the generations from LangChain to understand what is generated
print(result.generations[0][0].text)

```sql
-- Task: Select the name of the employee who has the highest salary
-- Generated by: SQLWhiz🤖

SELECT name
FROM employees
WHERE salary = (SELECT MAX(salary) FROM employees);
```


Great job! We have generated our very first SQL code using an LLM!

An improvement we can perform on this is using a **Few-Shot prompt** where we include a couple of examples for the LLM to understand what we are trying to do. Doing so should result in better results as the LLM has some reference examples about what we are trying to do.

**BUT** remember *we need to evaluate our LLM* prompt before we can even think about experimenting on it otherwise how would we even know if this new prompt is even better in the first place?

# 2. Evaluating LLMs on SQL Code Generation

Evaluating Text-to-SQL might seem like an easy tasks to evaluate as all we need in order to generate our Golden Dataset would be a list of natural language queries and their respective SQL query but there are some intricacies with it that will prove to be challenging.

We will need some way to check for **equivalence** to the **gold** query, which is where prior mentioned intricacies will start to pop up, but we will circle back to them later on. With that, we can start off with looking for an exact match between the LLM response and the golden query. For our initial metric we will be using a very simple metric, [exact match](https://huggingface.co/spaces/evaluate-metric/exact_match).

![Basic Exact Match LLM Evaluator](https://drive.google.com/uc?id=1yIygL2zNC4JZ02dMBtC5HyNa0_ujIOOv)

With a Golden Dataset which we can hand generate using our domain expertise, we can build up an evaluation dataset to run on every iteration to improve our application. Using the exact match metric we can derive an accuracy score of our LLM-application that we can keep track of and use in order to compare the performance across different iterations overtime.

Here is an example of what our dataset can look like:

![Example Golden Dataset](https://drive.google.com/uc?id=12D8C2juErtOzvKVLaKL_njxlceFJcbTu)

**NOTE:** If you noticed, there is a **complexity** column on the SQL code used in the example which we can try incorporating as an optional tasks in order to improve the quality of our results across different fine-grained categories which is paramount in ensuring our LLM SQL Code Generator is more robust. Ideally we should have an even distribution across all these additional categories that we add on, where we can then calculate metrics such as **Accuracy@HardComplexity** and so on.

We will provide you with a basic evaluation dataset to start off with, which we can further improve on using the process laid out below using [deepeval](https://github.com/confident-ai/deepeval):

![Evaluation Process](https://drive.google.com/uc?id=1KC_gMvU3PeflEm0N1Co8_YnT46lCvMLU)
<!-- 1. Load a Golden Test set for the students in a JSONL file with 20-30 examples
2. TODO: Create a custom deep-eval evaluator for SQL code where we check if the output of the SQL code exactly matches the output from the LLM.
3. TODO: Run evaluation on the golden test set.
4. TODO: Try different prompts and see if they improve the evaluation metric
5. Checking for an exact match is not the best way to go about it as the same query could have multiple ways to do it. Let's try using G-Eval where we use another more powerful LLM.
6. TODO: Evaluate using G-Eval
7. Talk about how in a Production system we should be running evaluation just like how we run automated tests, via CI/CD pipelines. -->

In [ ]:
BASIC_GROUND_TRUTH_DATASET = [
  {
    "Query": "Show the total number of employees.",
    "Ground Truth": "SELECT COUNT(*) AS total_employees FROM employees;"
  },
  {
    "Query": "Find the highest salary in the Finance department.",
    "Ground Truth": "SELECT MAX(salary) FROM employees WHERE department = 'Finance';"
  },
  {
    "Query": "Get the average age of all managers.",
    "Ground Truth": "SELECT AVG(age) FROM employees WHERE position = 'Manager';"
  },
  {
    "Query": "List the names and emails of staff in the IT department.",
    "Ground Truth": "SELECT name, email FROM employees WHERE department = 'IT';"
  },
  {
    "Query": "What are the titles of the top 5 selling books?",
    "Ground Truth": "SELECT title FROM books ORDER BY sales DESC LIMIT 5;"
  },
  {
    "Query": "Which product has the least quantity in stock?",
    "Ground Truth": "SELECT product_name FROM products ORDER BY quantity_in_stock ASC LIMIT 1;"
  },
  {
    "Query": "Display the second highest salary in the organization.",
    "Ground Truth": "SELECT MAX(salary) FROM employees WHERE salary < (SELECT MAX(salary) FROM employees);"
  },
  {
    "Query": "List employees who joined after 2015 and work in the Sales department.",
    "Ground Truth": "SELECT * FROM employees WHERE year(joined_date) > 2015 AND department = 'Sales';"
  },
  {
    "Query": "Find the average order value for each customer.",
    "Ground Truth": "SELECT customer_id, AVG(order_value) FROM orders GROUP BY customer_id;"
  },
  {
    "Query": "Show departments that have more than 10 employees.",
    "Ground Truth": "SELECT department FROM employees GROUP BY department HAVING COUNT(*) > 10;"
  },
  {
    "Query": "List all products that have never been ordered.",
    "Ground Truth": "SELECT * FROM products WHERE product_id NOT IN (SELECT product_id FROM orders);"
  },
  {
    "Query": "Which customers have spent more than $1000 in total?",
    "Ground Truth": "SELECT customer_id FROM orders GROUP BY customer_id HAVING SUM(order_value) > 1000;"
  },
  {
    "Query": "Show the total number of orders placed each day last week.",
    "Ground Truth": "SELECT order_date, COUNT(*) FROM orders WHERE order_date > CURRENT_DATE - INTERVAL '7 days' GROUP BY order_date;"
  },
  {
    "Query": "List the names of employees who do not manage anyone.",
    "Ground Truth": "SELECT name FROM employees WHERE name NOT IN (SELECT manager_name FROM employees);"
  },
  {
    "Query": "Display the department that has the highest average employee salary.",
    "Ground Truth": "SELECT department FROM employees GROUP BY department ORDER BY AVG(salary) DESC LIMIT 1;"
  }
]

In [ ]:
import pandas as pd

df = pd.DataFrame(BASIC_GROUND_TRUTH_DATASET)
df.head()

,Query,Ground Truth
0,Show the total number of employees.,SELECT COUNT(*) AS total_employees FROM employ...
1,Find the highest salary in the Finance departm...,SELECT MAX(salary) FROM employees WHERE depart...
2,Get the average age of all managers.,SELECT AVG(age) FROM employees WHERE position ...
3,List the names and emails of staff in the IT d...,"SELECT name, email FROM employees WHERE depart..."
4,What are the titles of the top 5 selling books?,SELECT title FROM books ORDER BY sales DESC LI...


In the next step, lets see how our LLM and Prompt are able to perform on the entire dataset by running it through and evaluating its performance. This is known as performing a LLM Task Evaluation which is different than LLM Model Evals. Task evaluation cares about the performance of the LLM on a specific task whereas Model evals care about the general performance of the LLM across a variety of tasks, benchmarks such as the [HellaSwag](https://rowanzellers.com/hellaswag/) dataset is an example of a Model Evals and Text2SQL accuracy like we are doing in this project is an example of task evaluation.

We will start off with traditional NLP metrics that compare the output with the ground truth can derive a score based on the exact/similarity of the two. These metrics are simple to use and serve as a good place to start off when evaluating LLMs.

Firstly, let's start off with the exact match metric which very simply just compares the output with the ground truth and checks whether its an exact match or not, deepeval implements these metrics within the [Scorer module](https://github.com/confident-ai/deepeval/blob/4b3ceed20993232331550798fe0a8f1bf2605594/deepeval/scorer/scorer.py#L7).

deepeval makes it easy to create our own metrics by inheriting the BaseMetric class, you can read more about creating custom metrics [here](https://docs.confident-ai.com/docs/metrics-custom).

In [ ]:
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase
from deepeval.scorer import Scorer


class ExactMatchMetric(BaseMetric):
    def __init__(self):
        super().__init__()
        self._threshold = 0.0

    def measure(self, test_case: LLMTestCase):
        self.success = Scorer.exact_match_score(
            test_case.actual_output, test_case.expected_output
        )
        if self.success:
            self.score = 1
        else:
            self.score = 0
        return self.score

    def is_successful(self):
        return bool(self.success)

    @property
    def __name__(self):
        return "ExactMatch"

In [ ]:
# Let's create our first test case!

test_case_input = df.head(1)
test_case_query = test_case_input['Query'].squeeze()
test_case_expected_output = test_case_input['Ground Truth'].squeeze()

In [ ]:
print(f"Here is our query: {test_case_query}")
print(f"Here is our ground truth: {test_case_expected_output}")

Here is our query: Show the total number of employees.
Here is our ground truth: SELECT COUNT(*) AS total_employees FROM employees;


### MY PROMPT

In [ ]:
ZERO_SHOT_PROMPT_TEMPLATE = """
# ROLE

Act as an Expert SQL Code Generator, specializing in taking a natural language {query} from a user and generating SQL Code based on the {query}.

# Context

You help users from a non-technical and a technical background to generate SQL code to get answers for their data queries faster in a reliable way.

Potential users are:
- **Project Managers**: They can use our tool to quickly derive important metrics for presentations or reports. Instead of relying on technical teams, they can directly query the database using natural language.
- **Data Scientists**: Often dealing with complex data queries, data scientists can use our tool to streamline and debug intricate SQL statements, saving time and reducing the potential for error.
- **Business Analysts and Non-Tech Professionals**: Anyone without deep SQL knowledge but needing to interact with the database can benefit immensely. They can easily convert their data needs into SQL queries without delving into the complexities of SQL syntax.

# RESPONSIBILITY

Generate SQL Code that helps the users with their Data needs to fulfill any requirements they have.
Create posts from “Quotes” that compels to audience to stop and gaze at the stunning image and as they whisper the quote to themselves and read the story, they smile and cannot help but feel motivated, inspired, and grateful as they come to acknowledge the mental resilience needed to go through life.

# INSTRUCTIONS
1. **Input:** User will enter a Natural Language “Query” for a specific task
2. Take your time to understand the query
3. Generate well organized and optimized SQL Code with concise and clear comments that fulfills the user requirements

# RULES
1. Write a comment what the task is about and also add your desgination - written by SQLWhiz🤖 on top of the generated SQL Code.
2. You only answer natural language queries in 'English'
3. The final output should be the SQL Code
"""

prompt = ZERO_SHOT_PROMPT_TEMPLATE.format(
    query=test_case_query
)

result = llm.generate([[HumanMessage(content=prompt)]])

test_case_actual_output = result.generations[0][0].text



In [ ]:
print(f"Here is what our model generated: {test_case_actual_output}")

Here is what our model generated: ```sql
-- Task: Show the total number of employees.
-- Generated by SQLWhiz🤖

SELECT COUNT(*) AS total_employees
FROM employees;
```


### PROMPT BY INSTRUCTOR



In [ ]:
ZERO_SHOT_PROMPT_TEMPLATE = """
Generate a valid SQL query for the following natural language instruction:

${query}

Only generate SQL code and nothing else.
"""

prompt = ZERO_SHOT_PROMPT_TEMPLATE.format(
    query=test_case_query
)

result = llm.generate([[HumanMessage(content=prompt)]])

test_case_actual_output = result.generations[0][0].text

print(f"Here is what our model generated: {test_case_actual_output}")

Here is what our model generated: SELECT COUNT(*) AS total_employees
FROM employees;


In [ ]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
  input=test_case_input,
  actual_output=test_case_actual_output,
  expected_output=test_case_expected_output
)

In [ ]:
exact_match_metric = ExactMatchMetric()
exact_match_metric.measure(test_case)

0

In [ ]:
print(f"Test case: {exact_match_metric.is_successful()}")

Test case: False


**NOTE:** If your prompt still fails and produces a different SQL code such as *"SELECT COUNT(*) AS total_employees FROM employees;"* feel free to update the ground truth to get it to pass in the meantime. This can happen as code correctness is not a simple task due to the fact that different implementations can still solve the query which is something we invite you to address via the optional tasks for this week.


Great job! We are able to see that our LLM and Prompt are able to answer the test case correctly, but that is just a basic example that we have. Next let us run it on the entire dataset that we have created.

## TODO: Create a deepeval dataset and evalute our LLM and Prompt on it

There is a slight difference between what we have been learning about Golden Datasets in general and how Golden Datasets work within deepeval. When we talk about a Golden Dataset as per the [course materials here](https://uplimit.com/course/llms-in-production/v2/module/evaluating-llms#corise_clrnjsvpd001m2e6iogarwdpn) we mention that its similar to a held-out test set meaning that we have data to predict on alongside their ground truth labels in order to derive evaluation metrics.

In deepeval, we will need to create an Evaluation Dataset instead where an Evaluation Dataset is built from LLMTestCase's and/or Golden's. Remember that the `actual_output` comes from running our LLM against the `input` as we did above!
```python
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset

first_test_case = LLMTestCase(input="...", actual_output="...")
second_test_case = LLMTestCase(input="...", actual_output="...")

test_cases = [first_test_case, second_test_case]
dataset = EvaluationDataset(test_cases=test_cases)
```
The definition of Golden's within deepeval is documented [here](https://docs.confident-ai.com/docs/confident-ai-manage-datasets#what-is-a-golden):
```
A "Golden" is what makes up an evaluation dataset and is very similar to a test case in deepeval, but they:

    - do not require an actual_output, so whilst test cases are always ready for evaluation, a golden isn't.
    - only exists within an EvaluationDataset(), while test cases can be defined anywhere.
    - contains an extra additional_metadata field, which is a dictionary you can define on Confident. Allows you to do some extra preprocessing on your dataset (eg., generating a custom LLM actual_output based on some variables in additional_metadata) before evaluation.

We introduced the concept of goldens because it allows you to create evaluation datasets on Confident without needing pre-computed actual_outputs. This is especially helpful if you are looking to generate responses from your LLM application at evaluation time.
```
With our EvaluationDataset created we can then run our LLM through it in order to derive the final score. For example, the following does so without using Pytest although its recommended that you treat this process as a CI/CD process that runs alongside your Pytest suite for your actual application.

```python
from deepeval import evaluate
from deepeval.metrics import HallucinationMetric, AnswerRelevancyMetric
from deepeval.dataset import EvaluationDataset

dataset = EvaluationDataset(test_cases=[...])
hallucination_metric = HallucinationMetric(threshold=0.3, context="...")
answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5)

dataset.evaluate([hallucination_metric, answer_relevancy_metric], run_async=False)

# You can also call the evaluate() function directly
evaluate(dataset, [hallucination_metric, answer_relevancy_metric], run_async=False)
```
An example output of the evaluation would look like this:
```
======================================================================

Metrics Summary

  - ✅ Answer Relevancy (score: 1, threshold: 0.5, strict: False, evaluation model: gpt-4-turbo, reason: The score is 1.00 because the response perfectly addresses the question about operating hours without any irrelevant information. Great job!, error: None)
  - ✅ Bias (score: 0, threshold: 0.5, strict: False, evaluation model: gpt-4-turbo, reason: The score is 0.00 because the output maintains neutrality and objectivity, as evidenced by the absence of any cited biased phrases or issues., error: None)

For test case:

  - input: What are your operating hours?
  - actual output: ...
  - expected output: None
  - context: ['Our company operates from 10 AM to 6 PM, Monday to Friday.', 'We are closed on weekends and public holidays.', 'Our customer service is available 24/7.']
  - retrieval context: None

======================================================================

Metrics Summary

  - ✅ Answer Relevancy (score: 1, threshold: 0.5, strict: False, evaluation model: gpt-4-turbo, reason: The score is 1.00 because the response accurately addresses the question about free shipping without any irrelevant information. Great job on maintaining focus!, error: None)
  - ✅ Bias (score: 0, threshold: 0.5, strict: False, evaluation model: gpt-4-turbo, reason: The score is 0.00 because the actual output perfectly demonstrates neutrality and objectivity, without any indication of bias., error: None)

For test case:

  - input: Do you offer free shipping?
  - actual output: ...
  - expected output: Yes, we offer free shipping on orders over $50.
  - context: None
  - retrieval context: None

======================================================================
```
One thing to note is that the metrics are run on a single (query, ground_truth) instead across the entire dataset meaning that we lack the ability to have a singular number to compare across different evaluation runs like traditional machine learning where you would for example have a accuracy score for the test set.

In our case, we will have to extend deepeval slightly to support this functionality and introduce a concept of pass rates which are similar to accuracy across the entire dataset but its just meant to show how many instances for a metric in the dataset passed a evaluation metric check.

With this you could enforce a strict pass rate on deployments much like how we can enforce Pytest code coverage to be off a certain %.


```python
from typing import List
from deepeval.evaluate import TestResult


def aggregate_metric_pass_rates(test_results: List[TestResult]):
    metric_counts = {}
    metric_successes = {}

    for result in test_results:
        for metric in result.metrics_metadata:
            metric_name = metric.__class__.__name__
            if metric_name not in metric_counts:
                metric_counts[metric_name] = 0
                metric_successes[metric_name] = 0
            metric_counts[metric_name] += 1
            if metric.success:
                metric_successes[metric_name] += 1

    metric_pass_rates = {metric: (metric_successes[metric] / metric_counts[metric]) for metric in metric_counts}

    print("\n" + "=" * 70 + "\n")
    print("Aggregate Metric Pass Rates\n")
    for metric, pass_rate in metric_pass_rates.items():
        print(f"{metric}: {pass_rate:.2%} pass rate")
    print("\n" + "=" * 70 + "\n")

    return metric_pass_rates
```
Running `aggregate_metric_pass_rates()` on the results of the evaluation give us pass rates across each metric that we use:

```python
evaluation_results = evaluate(dataset, [hallucination_metric, answer_relevancy_metric], run_async=False)
aggregate_metric_pass_rates(evaluation_results)
```
This should output the following where we get the pass rates across each metric and a mapping storing the pass rates across each metric which we can use to store and compare across different evaluation runs:
```
======================================================================

Aggregate Metric Pass Rates

AnswerRelevancyMetric: 100.00% pass rate
BiasMetric: 100.00% pass rate

======================================================================

{'AnswerRelevancyMetric': 1.0, 'BiasMetric': 1.0}
```

Now with this you can apply it to our dataset:
1. Create an EvaluationDataset using deepeval.
2. Run the evaluation on the Golden Dataset without Pytest, refer to [this](https://docs.confident-ai.com/docs/evaluation-introduction#evaluating-without-pytest).

In [ ]:
######################
# START YOUR CODE HERE
######################

Good job evaluating the LLM and Prompt on the entire dataset that we have created! If you notice it isn't that good at this task still. What do you think we can do to improve the evaluation performance? What issues do you see with the way we perform evaluations at this point of time?
- TODO: ADD YOUR THOUGHTS HERE

# 3. Validating LLM Outputs using Guardrails

When building our LLM Application one difficult thing is the fact that LLM's are non-deterministic in their outputs. We are able to control different parameters within our LLM to mitigate this to a certain extent such as:
1. temperature / top_p
2. seed

You can read more about these parameters from OpenAI's documentation [here](https://platform.openai.com/docs/api-reference/chat/create).

But ensuring consistent outputs is still a problem. *This is where having guardrails comes into play.*

> Guardrails = safety controls for LLMs

Just like guardrails help prevent cars from going off the road, they can also help out with making sure LLM outputs are in line with set expectations.

![Road guardrails](https://drive.google.com/uc?id=1KZCfJs9Sf6ExqpphnTa4G-AaisxFBRWG)

Guardrails can take many different forms which differ in complexity:
1. **Security Guardrails**, ensuring there isn't any PII data in the response
2. **Compliance Guardrails**, ensuring that competitors are not talked about
3. **Safety Guardrails**, ensuring that there are not toxic responses
4. **Structural Guardrails**, ensuring that the response matches a specific JSON schema


A simple Guardrail for this situation could be to ensure that there are **no default error results incorporated within the product description**.

In our application we will also be communicating via REST APIs which leverage JSON to send and receive data between different applications.

**Frontend JSON sent to the Backend:**
```json
{
    "prompt": "Write a SQL query to find the names and email addresses of all users in the 'users' table who joined after January 1, 2020.",
    "temperature": 0.1,
    "max_tokens": 100
}
```
**Backend Response JSON sent back to the Frontend:**
```json
{
  "id": "cmpl-XYZ456",
  "object": "text_completion",
  "created": 1616517999,
  "model": "gpt-3.5-turbo",
  "prediction": {
    "text": "SELECT name, email FROM users WHERE join_date > '2020-01-01';"
  }
}
```
The response from OpenAI will be used to populate the `prediction` field within the main Backend Response where we will have an object that contains a `text` field which will then be used to contain the actual output from OpenAI.

In this scenario we would initially look for a **Structural Guardrail** that ensures that the text response from the LLM:
1. *is a string*
2. *is not empty*

A great library for this would be [Guardrails.ai](https://www.guardrailsai.com/) which allows us to leverage [Pydantic](https://docs.pydantic.dev/latest/) in order to easily define validators for our LLM responses.


In [ ]:
from pydantic import BaseModel, Field

class LLMResponse(BaseModel):
  generated_sql: str = Field(description="Generated SQL from LLM")

In [ ]:
import openai
from rich import print
import guardrails as gd

# TODO: Add in your prompt here from before along with the query
# NOTE: gr.complete_json_suffix_v2 comes from guardrail where it will inject
#       additional tokens related to enforcing the Pydantic Model, to the main prompt.
ZERO_SHOT_PROMPT_GUARDRAILS_TEMPLATE = """
    ${query}

    ${gr.complete_json_suffix_v2}
"""

guard = gd.Guard.from_pydantic(output_class=LLMResponse, prompt=ZERO_SHOT_PROMPT_GUARDRAILS_TEMPLATE)

raw_llm_output, validated_output, *rest = guard(
    llm_api=openai.chat.completions.create,
    model=MODEL_NAME,
    prompt_params={
        "query": SAMPLE_QUERY
    },
)


In [ ]:
print(guard.base_prompt)

Generate a valid SQL query for the following natural language instruction:
    
    ${query}

    
Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="generated_sql" description="Generated SQL from LLM"/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

In [ ]:
print(raw_llm_output)

{
    "generated_sql": "SELECT name FROM employee ORDER BY salary DESC LIMIT 1"
}

In [ ]:
print(validated_output)

{'generated_sql': 'SELECT name FROM employee ORDER BY salary DESC LIMIT 1'}

In [ ]:
# We can view the history of our prompts in a nice tree like format
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │                                                                                                         │ │
    │ │     Generate a valid SQL query for the following natural language instruction:                          │ │
    │ │                                                                                                         │ │
    │ │     Select the name of the employee who has the highest salary                                          │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ Given below is XML that describes the information to extract from this document and the tags to extract │ │
    │ │ it into.                                                                                                │ │
    │ │                                                                                                         │ │
    │ │ <output>                                                                                                │ │
    │ │     <string name="generated_sql" description="Generated SQL from LLM"/>                                 │ │
    │ │ </output>                                                                                               │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ │ ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the │ │
    │ │ `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding  │ │
    │ │ XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g.        │ │
    │ │ requests for lists, objects and specific types. Be correct and concise.                                 │ │
    │ │                                                                                                         │ │
    │ │ Here are examples of simple (XML, JSON) pairs that show the expected behavior:                          │ │
    │ │ - `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`                     │ │
    │ │ - `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO',     │ │
    │ │ etc.]}`                                                                                                 │ │
    │ │ - `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index"          │ │
    │ │ format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`                        │ │
    │ │                                                                                                         │ │
    │ │                                                                                                         │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭───────────────────────────────────────────── Instructions ──────────────────────────────────────────────╮ │
    │ │ You are a helpful assistant, able to express yourself purely through JSON, strictly and precisely       │ │
    │ │ adhering to the provided XML schemas.                                                                   │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭────────────────────────────────────────────

## TODO: Adding More Guardrails
We have created a very basic Guardrail that leverages Pydantic Model's to ensure that our response fits a specifc format. Now let us try more advanced guardrails:
- Try creating **Bug Free SQL Code** leveraging the following [example](https://www.guardrailsai.com/docs/examples/syntax_error_free_sql).
- Feel free to add in more [validators](https://www.guardrailsai.com/docs/api_reference_markdown/validators) where you see fit for our use case. For example, we could ensure that there isn't any toxic language being returned from out LLM via using the [ToxicLanguage Validator](https://www.guardrailsai.com/docs/examples/toxic_language).
- How should we handle validation failures? One way would be to leverage reasking, where we reask the LLM to fix the output from a validation check failure. Within Guardrails.ai there are corrective measures we can take to handle validation errors as outlined [here](https://www.guardrailsai.com/docs/how_to_guides/rail#%EF%B8%8F-specifying-corrective-actions).




In [ ]:
######################
# START YOUR CODE HERE
######################

Great job adding in robust guardrails, next we can now focus on making sure our system performs at peak performance. Let us try improving the results of the model using Prompt Engineering techniques making sure that we compare results from the baseline with the new improvements. Feel free to go wild and experiment with all kinds of techniques for this section!

## TODO: Evaluation Driven Development(EDD)

EDD is similar to [Test Driven Development(TDD)](https://martinfowler.com/bliki/TestDrivenDevelopment.html) except we apply it to building LLM-powered applications, where our test cases in this case are derived from the Golden dataset that we created earlier and the goal here is to iteratively ensure that our LLM application can achieve a **100% pass rate** across all the examples.

The main idea here is that if we currently have a 60% pass rate on the exact match metric, we should take an iterative approach towards increasing the pass rate. For example, the LLM could be failing on the following query `Display the department that has the highest average employee salary.`. In this section we will try to tune the prompt or other parts to ensure that the model is able to correctly generate the ground truth for it.

NOTE: As we mentioned before some items from the dataset could result in false negatives since multiple implementations are possible for the same query.

Here are a couple of things we can try out:
- Try out Few-Shot Prompting and [other techniques](https://www.promptingguide.ai/techniques) and see how it impacts our evaluation performance.
- Try tuning the parameters used for a model.
- Try out different LLM models from OpenAI, Anthropic, etc.

You will have to setup an EvaluationDataset per each comparison since each run would have a different `actual_output` used within the LLMTestCase.
```python
dataset_run_1 = EvaluationDataset(test_cases=[...])
dataset_run_2 = EvaluationDataset(test_cases=[...])

evaluation_results_run_1 = evaluate(dataset_run_1, [exact_match_metric])
aggregate_metric_pass_rates(evaluation_results_run_1)

evaluation_results_run_1 = evaluate(dataset_run_1, [exact_match_metric])
aggregate_metric_pass_rates(evaluation_results_run_1)

assert evaluation_results_run_2[0].metrics_metadata[0].score > evaluation_results_run_1[0].metrics_metadata[0].score, "Run 2 does not perform better than Run 1"
```

In [ ]:
######################
# START YOUR CODE HERE
######################

## TODO: Evaluating SQL Code Using a Stronger LLM

Usually we would prioritise latency during inference using LLMs for most use cases, LLMs such as `gpt3.5-turbo` are much faster than `gpt4` variants while performing worse in general across benchmarks. What if we could also use a bigger and stronger LLM to aid with evaluations as well? There has been a rise of techniques doing just that where a smaller and faster LLM is evaluated by a bigger and stronger LLM. This process helps automate the evaluation part of any LLM-powered application! This is kinda of like having a dream within a dream just like in the movie *Inception*.

![Dream within a dream from Inception](https://drive.google.com/uc?id=1EC7rNg_LCLjIJ9SRyQpGVgtplOHZllDm)


**NOTE:** LLMs are very powerful and can help automate so many parts of the process but becareful not to be too reliant on them, at the end of the day they aren't perfect and hallucinate a lot. Therefore, its vital that you always keep a human-in-the-loop to verify things as a final seal of approval.


### Introducing G-Eval

[G-Eval](https://arxiv.org/abs/2303.16634) is a method that utilizes LLMs alongside a Chain-of-Thought (CoT) and form-filling approach to assess the outputs of LLMs. Initially, a task outline and evaluation parameters are provided to an LLM, which is then requested to create a CoT detailing the evaluation steps. For assessing the coherence of news summaries, the prompt, CoT, news article, and summary are merged, and the LLM is instructed to produce a score ranging from 1 to 5. Subsequently, the output token probabilities generated by the LLM are used to standardize this score, with a weighted summation taken as the ultimate outcome.

It was observed that using GPT-4 as an evaluator resulted in a high Spearman correlation with human assessments (0.514), surpassing all former methods. In the realm of summarization, it exceeded all prior state-of-the-art (SOTA) evaluators in the SummEval benchmark, which evaluates coherence, consistency, fluency, and relevance.


![G-Eval](https://drive.google.com/uc?id=1dq1BAjLY_Es5r-UiD96ZnFlxWTYcNv8g)

deepeval already comes out of the box with support for using G-Eval [here](https://docs.confident-ai.com/docs/metrics-llm-evals) where you can very easily just create a new metric and run it against your EvaluationDataset:

```python
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the expected output.",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)
```
Try creating a G-Eval metric for SQL Code Generation and running it through our EvaluationDataset like before.

In [ ]:
######################
# START YOUR CODE HERE
######################

Great job on getting G-Eval to work with our dataset! What do you think about using LLMs to evaluate another LLM vs just using traditional Machine Learning metrics? What are the Pros/Cons of it and where do you think it would work better or worse in?

**TODO: Add your thoughts here!**

# 4. Optional Tasks

- Try out implementing the Execution Score from the Snowflake team and comparing the results of it to Exact Match Metric and G-Eval, this [blog](https://medium.com/snowflake/inside-snowflake-building-the-most-powerful-sql-llm-in-the-world-1a33b3ee0d37) from the Snowflake Team covers how you can use LLMs to assists with evaluation.
- If you use SQL enough you would know that there are multiple ways in which we can answer a particular query especially if you had to update your ground truth earlier. With that you can try out other frameworks such as [sql-eval](https://github.com/defog-ai/sql-eval) which provides mo like we did before but more importanly also includes code execution into context when evaluating LLMs. T
- Guardrails do not just apply to the output from an LLM, instead we can also apply them to the actual input that a user feeds into the LLM. You can try out **Detecting Toxicity**, **Restricting the input to just be related to SQL code generation**, **Limiting the amount of tokens used**, etc.
- Our BASIC_GROUND_TRUTH_DATASET is quite small, see how you can add in more examples using your own domain knowledge or you could even leverage an LLM to do so via synthetic data generation which is supported within deepeval [here](https://docs.confident-ai.com/docs/evaluation-datasets-synthetic-data). Remember not all users will put in queries that can be answered with SQL code. For example, if the user asks `Who will win the next US Presidential Election?` you definitely don't want to answer it at all!
